#### Schätzung des ATTE mittels DoubleML unter Verwendung von Daten aus der Simulationstudie von Lee et al. (2011) und schauen, welche Auswirkungen die Discarding und Truncation Strategie auf die Schätzleistung von DoubleML haben. Dabei wird lineare Regression zur Vorhersage der Outcomes verwendet.

In [ ]:
# Notwendige Bibliotheken importieren
import numpy as np
import pandas as pd
import seaborn as sns
from doubleml import DoubleMLIRM, DoubleMLData
from doubleml.utils.resampling import DoubleMLResampling
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import  LinearRegression, LogisticRegressionCV 
from sklearn.ensemble import  RandomForestClassifier, GradientBoostingClassifier
from sklearn.base import clone
import warnings
warnings.filterwarnings("ignore")
import sys, os
# hier den Pfad zu der Funktion pscore_discard aus functions.py einfügen
sys.path.append(os.path.abspath("."))
from functions import pscore_discard
from joblib import Parallel, delayed

### DGP 

In [ ]:
# Funktion zur Erstellung einer kontinuierlichen Zufallsvariable, die mit der Variable x korreliert ist
def sample_cor(x, rho): 
    y = (rho * (x - np.mean(x))) / np.sqrt(np.var(x))+ np.sqrt(1 - rho**2) * np.random.randn(len(x))
    return y 

# Funktion zur Erstellung von Simulationsdaten
def generate_data(size, scenario):
    w1 = np.random.randn(size) #mean=0, sd=1
    w2 = np.random.randn(size)
    w3 = np.random.randn(size)
    w4 = np.random.randn(size)
    w5 = sample_cor(w1, 0.2)
    w6 = sample_cor(w2, 0.9)
    w7 = np.random.randn(size)
    w8 = sample_cor(w3, 0.2)
    w9 = sample_cor(w4, 0.9)
    w10 = np.random.randn(size)

    # Konvertieren der kontinuierlichen Variablen in binäre Variablen
    w1 = (w1 > np.mean(w1)).astype(int)
    w3 = (w3 > np.mean(w3)).astype(int)
    w5 = (w5 > np.mean(w5)).astype(int)
    w6 = (w6 > np.mean(w6)).astype(int)
    w8 = (w8 > np.mean(w8)).astype(int)
    w9 = (w9 > np.mean(w9)).astype(int)

    # Globale Koeffizienten definieren
    b0, b1, b2, b3, b4, b5, b6, b7 = 0, 0.8, -0.25, 0.6, -0.4, -0.8, -0.5, 0.7
    a0, a1, a2, a3, a4, a5, a6, a7, g1 = -3.85, 0.3, -0.36, -0.73, -0.2, 0.71, -0.19, 0.26, -0.4

    # Scenarien für die Erstellung der wahren Propensity Scores definieren
    if scenario == "A": #model with additivity and linearity
        z_a_trueps = 1 / (1 + np.exp(-(b0 + b1*w1 + b2*w2 + b3*w3 + b4*w4 + b5*w5 + b6*w6 + b7*w7)))
    elif scenario == "E": #mild non-additivity and non-linearity
        z_a_trueps = 1 / (1 + np.exp(-(b0 + b1*w1 + b2*w2 + b3*w3 + b4*w4 + b5*w5 + b6*w6 + b7*w7 + b2*w2*w2 + b1*0.5*w1*w3 + b2*0.7*w2*w4 + b4*0.5*w4*w5 + b5*0.5*w5*w6)))
    else:  # scenario G - moderate non-additivity and non-linearity
        z_a_trueps = 1 / (1 + np.exp(-(b0 + b1*w1 + b2*w2 + b3*w3 + b4*w4 + b5*w5 + b6*w6 + b7*w7 + b2*w2*w2 + b4*w4*w4 + b7*w7*w7 + b1*0.5*w1*w3 + b2*0.7*w2*w4 + b3*0.5*w3*w5 + b4*0.7*w4*w6 + b5*0.5*w5*w7 + b1*0.5*w1*w6 + b2*0.7*w2*w3 + b3*0.5*w3*w4 + b4*0.5*w4*w5 + b5*0.5*w5*w6)))

    # Wahrscheinlichkeit von Behandlungszuweisung
    prob_exposure = np.random.rand(size)
    z_a = np.where(z_a_trueps > prob_exposure, 1, 0)

    # Outcome-Variable modellieren und Zufallsrauschen hinzufügen
    y_a = a0 + a1*w1 + a2*w2 + a3*w3 + a4*w4 + a5*w8 + a6*w9 + a7*w10 + g1*z_a + np.random.randn(size)

    # Simulationsdaten erstellen
    sim_data = pd.DataFrame({
        'w1': w1, 'w2': w2, 'w3': w3, 'w4': w4, 'w5': w5, 
        'w6': w6, 'w7': w7, 'w8': w8, 'w9': w9, 'w10': w10,
        'z_a': z_a, 'y_a': y_a, 'z_a_trueps': z_a_trueps
    })
    return sim_data

### Truncation

In [ ]:
# Seed für die Reproduzierbarkeit festlegen
np.random.seed(1234)
# Schätzen die Propensity Score und den ATTE für verschiedene Schwellenwerte, Scenarien und Machine Learning-Algorithmen
def run_trunc (i, ml_g, ml_m, scenario):
    # Liste für die Ergebnisse erstellen
    results_tr = []
    # Seed festlegen
    np.random.seed(i)
    # Simulationsdaten erstellen
    simdata = generate_data(500, scenario)
    # wahren Propensity Score speichern
    trueps = simdata['z_a_trueps']
    # DoubleMLData Objekt erstellen
    data = DoubleMLData(simdata, y_col='y_a', d_cols='z_a', x_cols=['w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10'])          
    for trim_value in [0.001, 0.01, 0.05, 0.1]:
        # DoubleML Objekt erstellen
        dml_obj = DoubleMLIRM(data, ml_g, ml_m, trimming_threshold=trim_value, score="ATTE")
        # Schätzen der Effekte
        dml_obj.fit()
        # geschätzte Propensity Score speichern
        predicted_ps = dml_obj.predictions['ml_m'].flatten()
        dml_summary = dml_obj.summary
        dml_summary['scenario'] = scenario
        dml_summary['trim_value'] = trim_value
        dml_summary['learner'] = str(ml_m)
        # Anteil der trunkierten Beobachtungen und Anteil der behandelten Beobachtungen berechnen
        dml_summary['share_trimmed_top'] = (((predicted_ps == (1-trim_value)).sum()) / len(simdata))*100
        dml_summary['share_trimmed_bottom'] = (((predicted_ps == trim_value).sum()) / len(simdata))*100
        dml_summary['share_treated'] = (((simdata.loc[simdata['z_a'] == 1]).sum()) / len(simdata))*100
        # Anteil der trunkierten Beobachtungen mit wahren PS   
        dml_summary['share_trim_orcl_top'] = ((np.where(trueps >= (1-trim_value), 1, 0).sum())/trueps.shape[0])*100
        dml_summary['share_trim_orcl_bottom'] = ((np.where(trueps <= trim_value, 1, 0).sum())/trueps.shape[0])*100 
        
        # Nuisance Loss berechnen
        dml_summary['loss_ml_g0'] = dml_obj.nuisance_loss['ml_g0'][0][0]
        dml_summary['loss_ml_g1'] = dml_obj.nuisance_loss['ml_g1'][0][0]
        dml_summary['loss_ml_m'] = dml_obj.nuisance_loss['ml_m'][0][0]

        # Daten
        true_data_z_a_0 = simdata[simdata['z_a'] == 0]
        true_data_z_a_1 = simdata[simdata['z_a'] == 1]
        pred_g0 = dml_obj.predictions['ml_g0'].flatten()
        pred_g1 = dml_obj.predictions['ml_g1'].flatten()
        
        # RMSE für das Outcome Modell -> ml_g0, ml_g1 und Log Loss für das PS Modell -> ml_m berechnen
        dml_summary['loss_g0'] = np.sqrt(np.mean((true_data_z_a_0['y_a'] - pred_g0[true_data_z_a_0.index])**2)).round(6)
        dml_summary['loss_g1'] = np.sqrt(np.mean((true_data_z_a_1['y_a'] - pred_g1[true_data_z_a_1.index])**2)).round(6)
        dml_summary['loss_m'] = -np.mean(simdata['z_a'] * np.log(predicted_ps) + (1 - simdata['z_a']) * np.log(1 - predicted_ps)).round(6)

        # Ergebnisse speichern
        results_tr.append(dml_summary)
    return results_tr

# Parameter festlegen
learner_list = [
    {"ml_g": LinearRegression(), "ml_m": LogisticRegressionCV()},
    {"ml_g": LinearRegression(), "ml_m": RandomForestClassifier()},
    {"ml_g": LinearRegression(), "ml_m": GradientBoostingClassifier()}]
scenario_list = ['A', 'E', 'G']

# Simulationen parallel ausführen
result_trunc = {}
for scenario in scenario_list:
    result_trunc_scenario = {}
    for i_learner, learner in enumerate(learner_list):
        result = Parallel(n_jobs=-1)(delayed(run_trunc)(i, ml_g=clone(learner['ml_g']), ml_m=clone(learner['ml_m']), scenario=scenario) for i in range(1000))
        result_trunc_scenario[i_learner] = result
    result_trunc[scenario] = result_trunc_scenario

In [ ]:
# die absolute Verzerrung berechnen und prüfen, ob der wahre ATTE in den Konfidenzintervallen enthalten ist
true_atte = -0.4
result_trunc2 = {}
for scenario in scenario_list:
    result_scnr = result_trunc[scenario]
    result_trunc_scnr2 = {}
    for i_learner, learner in enumerate(learner_list):
        res = result_scnr [i_learner]
        combined_dfs_trunc = [pd.concat(inner_list, ignore_index=True) for inner_list in res]
        results = pd.concat(combined_dfs_trunc, ignore_index=True)
        results['bias'] = results['coef'] - true_atte
        results['abs_bias'] = np.abs(results['coef'] - true_atte) 
        results['in_ci'] = np.where((true_atte >= results['2.5 %']) & (true_atte <= results['97.5 %']), 1, 0)
        result_trunc_scnr2[i_learner] = results
    result_trunc2[scenario] = result_trunc_scnr2

In [108]:
# Speichern die Ergebnisse in CSV-Dateien
for scenario in scenario_list:
    result_scnr = result_trunc2[scenario]
    results = pd.concat([result_scnr[0], result_scnr[1], result_scnr[2]], ignore_index=True)
    results.to_csv(f"results_trunc_{scenario}.csv", index=False)

### Discarding

In [ ]:
# Seed für die Reproduzierbarkeit festlegen
np.random.seed(1234)

# Schätzen die Propensity Score und den ATTE für verschiedene Schwellenwerte, Scenarien und Machine Learning Algorithmen
def run_discard(i, ml_g, ml_m, scenario):
    # Liste für die Ergebnisse erstellen
    result_discard = []
    # Seed festlegen
    np.random.seed(i)
    # Simulationsdaten erstellen
    df_data = generate_data(500, scenario)
    # wahren Propensity Score speichern
    trueps = df_data['z_a_trueps']
    # Cross-Fitting
    resampling_obj = DoubleMLResampling(n_folds=5, n_rep=1, n_obs=len(df_data), stratify=df_data.z_a)
    smpls = resampling_obj.split_samples()

    # Propensity Scores schätzen
    pscore_est = cross_val_predict(ml_m, df_data.drop(['z_a', 'y_a'], axis=1), df_data.z_a, method='predict_proba', cv=resampling_obj.resampling)[:,1]

    for trim_value in [0.001, 0.01, 0.05, 0.1]:
        smpls_new, data_trimmed, pscore_trimmed = pscore_discard(df_data, pscore_est, smpls, trim_value)

        # DoubleMLData Objekt erstellen
        dml_data = DoubleMLData.from_arrays(x=data_trimmed.drop(columns = ['z_a', 'y_a']), y=data_trimmed.y_a, d=data_trimmed.z_a)
        # DoubleML Objekt erstellen
        dml_obj = DoubleMLIRM(dml_data, ml_g, ml_m, trimming_threshold=1e-12, draw_sample_splitting=False, score="ATTE")
        dml_obj.set_sample_splitting(smpls_new)
        dml_obj.fit(external_predictions={"d":{"ml_m": pscore_trimmed}})
        dml_summary = dml_obj.summary
        dml_summary['scenario'] = scenario
        dml_summary['trim_value'] = trim_value
        dml_summary['learner'] = str(ml_m)
        # Anteil der discarded Beobachtungen
        dml_summary['share_trimmed_top'] = ((np.where(pscore_est >= (1-trim_value), 1, 0).sum())/pscore_est.shape[0])*100
        dml_summary['share_trimmed_bottom'] = ((np.where(pscore_est <= trim_value, 1, 0).sum())/pscore_est.shape[0])*100
        # Anteil der Beobachtungen in der Behandlungsgruppe in der Gesamtstichprobe und in der Stichprobe nach dem Discarding
        dml_summary['share_treated_pop'] = ((df_data['z_a']== 1).sum()/df_data.shape[0])*100  
        dml_summary["share_treated_sample"] = ((data_trimmed['z_a'] == 1).sum()/data_trimmed.shape[0])*100  
        # Anteil der discarded Beobachtungen mit wahren PS   
        dml_summary['share_trim_orcl_top'] = ((np.where(trueps >= (1-trim_value), 1, 0).sum())/trueps.shape[0])*100
        dml_summary['share_trim_orcl_bottom'] = ((np.where(trueps <= trim_value, 1, 0).sum())/trueps.shape[0])*100 

        # Nuisance Loss berechnen
        dml_summary['loss_ml_g0'] = dml_obj.nuisance_loss['ml_g0'][0][0]
        dml_summary['loss_ml_g1'] = dml_obj.nuisance_loss['ml_g1'][0][0]
        dml_summary['loss_ml_m'] = dml_obj.nuisance_loss['ml_m'][0][0]
        
        # Daten
        df_data_trimmed = df_data.loc[data_trimmed.index].reset_index(drop=True)
        data_trimmed_z_a_0 = df_data_trimmed[df_data_trimmed['z_a'] == 0]
        data_trimmed_z_a_1 = df_data_trimmed[df_data_trimmed['z_a'] == 1]
        pred_g0 = dml_obj.predictions['ml_g0'].flatten()
        pred_g1 = dml_obj.predictions['ml_g1'].flatten()
        predicted_ps = pscore_trimmed.flatten()

        # RMSE für das Outcome Modell -> ml_g0, ml_g1  und Log Loss für das PS Modell -> ml_m berechnen
        dml_summary['loss_g0'] = np.sqrt(np.mean((data_trimmed_z_a_0['y_a'] - pred_g0[data_trimmed_z_a_0.index])**2)).round(6)
        dml_summary['loss_g1'] = np.sqrt(np.mean((data_trimmed_z_a_1['y_a'] - pred_g1[data_trimmed_z_a_1.index])**2)).round(6)
        dml_summary['loss_m'] = -np.mean(df_data_trimmed['z_a'] * np.log(predicted_ps) + (1 - df_data_trimmed['z_a']) * np.log(1 - predicted_ps)).round(6)    

        # Ergebnisse speichern
        result_discard.append(dml_summary)    
    return result_discard

# Parameter festlegen
learner_list = [
    {"ml_g": LinearRegression(), "ml_m": LogisticRegressionCV()},
    {"ml_g": LinearRegression(), "ml_m": RandomForestClassifier()},
    {"ml_g": LinearRegression(), "ml_m": GradientBoostingClassifier()}]
scenario_list = ['A', 'E', 'G']

# Simulationen parallel ausführen
result_discard = {}
for scenario in scenario_list:
    result_discard_scnr = {}
    for i_learner, learner in enumerate(learner_list):
        result = Parallel(n_jobs=-1)(delayed(run_discard)(i, ml_g=clone(learner['ml_g']), ml_m=clone(learner['ml_m']), scenario=scenario) for i in range(1000))
        result_discard_scnr[i_learner] = result
    result_discard[scenario] = result_discard_scnr

In [ ]:
# Die absolute Verzerrung berechnen und prüfen, ob der wahre ATTE in den Konfidenzintervallen enthalten ist
true_atte = -0.4
result_discard2 = {}
for scenario in scenario_list:
    result_scnr = result_discard[scenario]
    result_discard_scnr2 = {}
    for i_learner, learner in enumerate(learner_list):
        res = result_scnr[i_learner]
        combined_dfs_discard = [pd.concat(inner_list, ignore_index=True) for inner_list in res]
        results_discard = pd.concat(combined_dfs_discard, ignore_index=True)
        results_discard['bias'] = results_discard['coef'] - true_atte
        results_discard['abs_bias'] = np.abs(results_discard['coef'] - true_atte) 
        results_discard['in_ci'] = np.where((true_atte >= results_discard['2.5 %']) & (true_atte <= results_discard['97.5 %']), 1, 0)
        result_discard_scnr2[i_learner] = results_discard
    result_discard2[scenario] = result_discard_scnr2	

In [ ]:
# Speichern die Ergebnisse in CSV-Dateien
for scenario in scenario_list:
    result_scnr = result_discard2[scenario]
    results = pd.concat([result_scnr[0], result_scnr[1], result_scnr[2]], ignore_index=True)
    results.to_csv(f"results_disc_{scenario}.csv", index=False)